# Initialization 

In [107]:
import cobra
from cobra.io import load_model
import cometspy as c
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools

E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")
S0_ac = cobra.io.read_sbml_model("./models/STM_v1_0_S0.xml")
S0_glc = S0_ac.copy()

In [108]:
# E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")
lactose_medium = {'EX_ca2_e': 10,
 'EX_cl_e': 10,
 'EX_cobalt2_e': 10,
 'EX_cu2_e': 10,
 'EX_fe2_e': 10,
 'EX_fe3_e': 10,
 'EX_k_e': 10,
 'EX_mg2_e': 10,
 'EX_mn2_e': 10,
 'EX_mobd_e': 10,
 'EX_ni2_e': 10,
 'EX_o2_e': 10,
 'EX_pi_e': 10,
 'EX_so4_e': 10,
 'EX_zn2_e': 10,
 'EX_nh4_e': 10,
 'EX_lcts_e': 10}
lactose_met_medium = lactose_medium.copy()
lactose_met_medium["EX_met__L_e"] = 10 
E0.medium = lactose_met_medium 
E0.obj_style = "MAX_OBJECTIVE_MIN_TOTAL" 

def change_medium(model, metabolite, value): # function for setting medium metabolite value
    medium = model.medium
    medium[metabolite] = value
    model.medium = medium

ac_medium = {'EX_ca2_e': 10,
 'EX_cl_e': 10,
 'EX_cobalt2_e': 10,
 'EX_cu2_e': 10,
 'EX_fe2_e': 10,
 'EX_fe3_e': 10,
 'EX_k_e': 10,
 'EX_mg2_e': 10,
 'EX_mn2_e': 10,
 'EX_mobd_e': 10,
 'EX_ni2_e': 10,
 'EX_o2_e': 10,
 'EX_pi_e': 10,
 'EX_so4_e': 10,
 'EX_zn2_e': 10,
 'EX_nh4_e': 10,
 'EX_ac_e': 10}
S0_ac.medium = ac_medium 

S0_glc.medium = ac_medium
change_medium(S0_glc, 'EX_ac_e', 0)
change_medium(S0_glc, 'EX_glc__D_e', 10)
S0_ac.obj_style = "MAX_OBJECTIVE_MIN_TOTAL" 
S0_glc.obj_style = "MAX_OBJECTIVE_MIN_TOTAL" 




In [109]:
import sys
sys.path.append("..")
# sys.path.append("../../")
S2tab = pd.read_excel('S2Table.xlsx');
S2_Enzyme = list(S2tab['Enzyme'])
essential_genes = ['purU', 'pyrE', 'thrB', 'yrbG', 'folA', 'folP', 'pykF', 'rffG']
FVA_bounds = pd.read_csv('FVA_bounds_full.csv', index_col= 0)

D:\anaconda3\envs\Flux\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


remove reactions involved in preexisting gene inhibition candidates

In [110]:
def get_gene_id(model, gene_name):
    for _,i in enumerate(model.genes):
        if(i.name == gene_name):
            return(i.id)

for i, Enzyme in enumerate(S2_Enzyme):
    mark_remove = False
    for _, current_gene in enumerate(essential_genes):
        for _, current_reaction in enumerate(E0.genes.get_by_id(get_gene_id(E0,current_gene)).reactions):           
            if (Enzyme==current_reaction.id):
                mark_remove = True
                print(current_reaction.id)
    if (mark_remove == True):
        S2_Enzyme.remove(Enzyme)            
            

DHPS2
DHFR
PYK


Using Reactions
Only the first gene in case of degeneracy and choose only genes that also exist in S.enterica

In [111]:
# return first common gene name        
def extract_gene_name(gene_str, model):
    if (model.id =='iML1515'):
        offset = 5
        pos = [pos for pos, char in enumerate(gene_str) if char == 'b']
    else:
        offset = 7
        pos = [pos for pos, char in enumerate(gene_str) if char == 'S']

    pos.reverse()
    gene_list = list()
    gene_str_temp = gene_str+' '
    for i,val in enumerate(pos):
        gene_id = gene_str[val:val+offset]
        gene_list.append(model.genes.get_by_id(gene_id).name)
#         print(gene_id)
        gene_str_temp = gene_str_temp[:val] + model.genes.get_by_id(gene_id).name + gene_str_temp[val+offset:]
    gene_list.reverse()
    return(gene_list,gene_str_temp)

def get_common_gene(Enzyme):
    Enzyme = Enzyme.replace('-','_')
    try: 
        gpr_S0 = str(S0_ac.reactions.get_by_id(Enzyme).gpr)
        gpr_E0 = str(E0.reactions.get_by_id(Enzyme).gpr)
        gene_list_S0,_ = extract_gene_name(gpr_S0, S0_ac)
        gene_list_E0,_ = extract_gene_name(gpr_E0, E0)

        desired_gene = next((ele for ele in gene_list_E0 if ele in set(gene_list_S0)), None) # first occurrence
        return(desired_gene, gene_list_E0, gene_list_S0)
    except:
        print(f'{Enzyme} does not co-exist in E.coli and S.enterica') 
        return(None,(),())

In [112]:
desired_genes = list()
none_common_gene_mat = list()
for _,Enzyme in enumerate(S2_Enzyme):
    Enzyme = Enzyme.replace('-','_')
#     print(Enzyme)
    gene_to_append,b,b = get_common_gene(Enzyme)
    if(gene_to_append == None):
        none_common_gene_mat.append(Enzyme)
    else:
        desired_genes.append(gene_to_append)
        
desired_genes = set(desired_genes)
potential_genes = desired_genes | set(essential_genes)   

GLUDC does not co-exist in E.coli and S.enterica


In [113]:
# Perform gene inhibition
def set_bound(model, current_reaction, bound):
    old_bound = model.reactions.get_by_id(current_reaction).bounds
    if (bound>=0):
        ub = bound
        lb = 0 if old_bound[0]==0 else -bound
    else:
        lb = bound
        ub = 0 if old_bound[1]==0 else -bound 
    model.reactions.get_by_id(current_reaction).bounds = (lb,ub)
    
    
obj_flux_df = ['Normal']
obj_flux_df.extend([E0.slim_optimize()])
obj_flux_df.extend([S0_ac.slim_optimize()])
obj_flux_df.extend([S0_glc.slim_optimize()])
obj_flux_df = pd.DataFrame(obj_flux_df).T

# obj_flux_df = pd.concat([obj_flux_df, ], axis = 0)
# for _,current_gene in enumerate(potential_gene):
for _,current_gene in enumerate(list(potential_genes)[:]):
    new_row = list([current_gene])
    for _, (model, FVA_min) in enumerate(zip([E0,S0_ac,S0_glc],
                                          [FVA_bounds[['FVA_E0']],
                                          FVA_bounds[['FVA_S0_ac']],
                                          FVA_bounds[['FVA_S0_glc']]])):
        with model:
            for _,current_reaction in enumerate(model.genes.get_by_id(get_gene_id(model, current_gene)).reactions):
                set_bound(model, current_reaction.id, FVA_min.loc[current_reaction.id][0]/2)
            obj_val = model.slim_optimize()
            new_row.extend([obj_val])
    obj_flux_df = pd.concat([obj_flux_df, pd.DataFrame(new_row).T], axis=0)
                
obj_flux_df.columns = ['Gene_inhibition', 'E0', 'S0_ac', 'S0_glc']
obj_flux_df = obj_flux_df.set_index('Gene_inhibition')
%store obj_flux_df
obj_flux_df.to_csv('obj_flux_df.csv')

Stored 'obj_flux_df' (DataFrame)


# Functions for getting objective values

In [127]:
a = 5.1
type(a) is int or type(a) is float

True

In [114]:
alpha,obj_val = 1,0 
def separate_reaction(model, reaction_id, forward_terminal_change = True):
    """
    decompose reversible reaction into forward and backward
    forward_terminal_change : classification of whether the forward reaction aligns with sign of FVA  
    if direction aligns, terminal metabolite is to be scaled -> original reaction
    if direction is opposite, quantity of both substrate and product in reaction is kept intact -> reaction_v1
    """
    
    (lb, ub) = model.reactions.get_by_id(reaction_id).bounds 
    rct_ids = [reaction_id] #? 
    if(lb < 0 and ub !=0): # only perform if reaction is bidirectional
        intact_reaction = model.reactions.get_by_id(reaction_id).copy() # copy of target reaction

        if (forward_terminal_change ==True): # forward reaction matches sign of FVA bound 
            model.reactions.get_by_id(reaction_id).bounds = (0,ub) # forward only for the orginal reaction
            intact_reaction.bounds = (lb,0) # backward only for reaction_v1
        else: # forward reaction is opposite of sign of FVA bound 
            model.reactions.get_by_id(reaction_id).bounds = (lb,0) 
            intact_reaction.bounds = (0,ub)

        intact_reaction.id = f'{reaction_id}_v1' # redefine id for copy of reaction
        model.add_reactions([intact_reaction]) # add to model
        rct_ids.append(intact_reaction.id) # list of id for reaction and reaction_v1 
#         print(rct_ids)
    return(rct_ids)

def convert_arg_to_list(arg):
    if type(arg) is not list and type(arg) is not tuple:
        arg = [arg]
    return(arg)

def get_summary_df(model = E0, alpha = alpha, obj_val = obj_val, rct_ids = 'DHFR', sol_sol = None): 
#     summarize solutions from optimization and alpha used  
# expect direction opposite with zero flux, to be consistent with FVA bound 
    if type(alpha) != int and type(alpha) != float:
        alpha = str(alpha)
    if sol_sol is not None:
        rct_dir, rct_rev, flux_dir, flux_rev = list(), list(), list(), list()
        for current_rct_id in rct_ids:
            append_str = str(model.reactions.get_by_id(current_rct_id))
            if ('_v1' not in current_rct_id): # direction align with FVA
                rct_dir.extend([append_str])
                try:
                    flux_dir.extend([round(model.reactions.get_by_id(current_rct_id).flux,5)])
                except: 
                    flux_dir.extend([sol_sol[current_rct_id]])
            else: # direction opposite
                rct_rev.extend([append_str])        
                try:
                    flux_rev.extend([round(model.reactions.get_by_id(current_rct_id).flux,5)])
                except:
                    flux_rev.extend([sol_sol[current_rct_id]])
    #     rct_rev = fix_string(rct_rev)
    #     rct_dir = fix_string(rct_dir)
        net_flux = sum(abs(element) for element in set(flux_dir) | set(flux_rev))
        net_flux_I = 'Zero Flux' if net_flux ==0 else 'Net Flux'

    #     if type(alpha) is list or type(alpha) is tuple:

        summary_df = pd.DataFrame({f'div_opt_alpha': alpha,
                                   f'div_opt_obj_val': obj_val,
                                   f'FVAdir_Reactions_id': ', '.join(rct_ids),
                                   f'FVAdir_Reactions_involved': ' '.join(rct_dir),                                    
                                   f'FVAdir_Flux_values':[flux_dir],
                                   f'FVAopposite_Reactions_involved': ', '.join(rct_rev),     
                                   f'FVAopposite_Flux_values':[flux_rev],
                                   f'Net_Flux_Boolean':[net_flux_I],
                                   f'Net_Flux':[net_flux]}
                                 )
#     else:
#         summary_df = pd.DataFrame([{'div_opt_obj_val': obj_val}])        
    return(summary_df)

# def swap_gene_pair_order(alphas, genes,exception=500): # larger alpha come first 
#     if len(alphas)>1:  
#         cond = ((alphas[1]>alphas[0] or alphas[0] == exception) and # keep larger alpha in front, except for nonessential genes
#             alphas[1] != exception)                           
#         alphas, genes= list(reversed(alphas)), list(reversed(genes))
#     return(alphas, genes)

def Sij_obj(model = E0, alphas = 1, genes = 'folA', flux_df_col = 'E0', slim_opt = False): 
    # get objective value for corresponding alpha
    rct_ids = list() # store list of id of reaction and reaction_v1 that regulated by the same gene 
    with model:  # enumerate genes
        for current_gene, alpha in zip(convert_arg_to_list(genes), convert_arg_to_list(alphas)):
#             print(current_gene, alpha)
            for rct in model.genes.get_by_id(get_gene_id(model, current_gene)).reactions: 
#                 print(rct)
    #             print(model.reactions.get_by_id(rct.id))
                try:
                    if(FVA_bounds[f'FVA_{flux_df_col}'][rct.id]>=0):   # if forward reaction align with FVA, reduce product produced
                        rct_ids.extend(separate_reaction(model, rct.id, forward_terminal_change=True))# copy of reaction, forward_terminal_change = True
                        dict_product = dict((k, v) for k, v in model.reactions.get_by_id(rct.id).metabolites.items() if v >= 0)
                    else:  # forward reaction oppose to FVA, reduce substrate consumed
                        rct_ids.extend(separate_reaction(model, rct.id, forward_terminal_change=False))# copy of reaction, forward_terminal_change = True           
                        dict_product = dict((k, v) for k, v in model.reactions.get_by_id(rct.id).metabolites.items() if v <= 0)
                    for product, unit in dict_product.items():  # scale corresponding metabolite units involved in the reaction
                        model.reactions.get_by_id(rct.id).add_metabolites({product: -unit*(1-1/alpha)}) # only change unit of unidirection metabolite
                except: 
                    print(f'{rct.id} duplicated reaction')    
                    # without abs, desired metabolites approach 0(Less -, less +)
        if (slim_opt == False):
            sol_sol = model.optimize()    
            obj_val = sol_sol.objective_value     
            summary_df = get_summary_df(model, alphas, obj_val, rct_ids, sol_sol.fluxes)  
            return(alpha, obj_val, summary_df)            
        else: 
#             return(pd.DataFrame([{f'div_opt_obj_val': model.slim_optimize()}]))
            return(model.slim_optimize())        

functions for searching alpha

In [115]:
def evaluate_alpha(model, search_alpha, current_gene, flux_df_col, target_obj_val, opt_df, alpha_lb , alpha_ub, exp_leap, precision):
    _, obj_val, summary_df = Sij_obj(model, search_alpha, current_gene, flux_df_col)

    if(opt_df is None and summary_df['Net_Flux'][0]<1e-8): # reinitialize if the first alpha to search gives zero flux
        search_alpha = 1+2e-3
        _, obj_val, summary_df = Sij_obj(model, search_alpha, current_gene, flux_df_col)
        
    halt_condition = (summary_df['Net_Flux'][0]<1e-8 or summary_df['Net_Flux'][0]>100 or 
                      obj_val<target_obj_val or obj_val>1)
    termination_condition = (round(search_alpha,precision)==round(alpha_ub,precision) or
                             round(search_alpha,precision)==round(alpha_lb,precision) or
                            search_alpha>alpha_ub-1)
    # update optimal df
    if((summary_df['Net_Flux'][0]>0 and 
        halt_condition == False) or
       (opt_df is None or 
       summary_df['div_opt_obj_val'][0] > target_obj_val*0.9)):
        opt_df = summary_df

    return(search_alpha, termination_condition, halt_condition, opt_df)

def find_feasible_alpha(model, search_alpha, current_gene, flux_df_col = 'E0', target_obj_val = 0.5,
                        alpha_lb = 1+1e-7, alpha_ub = 1e5, exp_leap = 2, precision = 3, low_alpha_label = False):
    opt_df = None # store optimal summary for return function     
    search_alpha, termination_condition, halt_condition,opt_df = evaluate_alpha(model, search_alpha, current_gene, flux_df_col, target_obj_val, opt_df, alpha_lb , alpha_ub, exp_leap, precision)   
#     print(search_alpha,termination_condition, halt_condition)
    while (termination_condition == False): 
        if (halt_condition == True): # search alpha not accepted, adjust search towards alpha_lb
            # start binary search, in (alpha_lb, search_alpha)
            alpha_lb = alpha_lb
            alpha_ub = search_alpha
            
            search_alpha = max(min(1+(alpha_lb-1)*2,(search_alpha+alpha_lb)/2),1e-5)
            
#             if(low_alpha_label == True): 
#                 search_alpha = max(min(1+(alpha_lb-1)*2,(search_alpha+alpha_lb)/2),1e-5)
                
# #                 print('label activated',1+(alpha_lb-1)*2,(search_alpha+alpha_lb)/2)
#             else:
#                 search_alpha = (search_alpha+alpha_lb)/2
# #             print(halt_condition, alpha_lb, search_alpha, alpha_ub)
            _, termination_condition, halt_condition, summary_df = evaluate_alpha(model, search_alpha, current_gene, flux_df_col, target_obj_val, opt_df, alpha_lb , alpha_ub, exp_leap, precision)
            
        else: # press alpha_ub
#             print(summary_df)
            alpha_lb = search_alpha
            
            if(search_alpha*exp_leap <alpha_ub): # exponential search(large step)
                while (search_alpha*exp_leap <alpha_ub and halt_condition == False): 
        #             print('Exp search')
                    search_alpha *= exp_leap
                    _, termination_condition, halt_condition,summary_df = evaluate_alpha(model, search_alpha, current_gene, flux_df_col, target_obj_val, opt_df, alpha_lb , alpha_ub, exp_leap, precision)
            else: 
                # binary search (small step), adjust towards alpha_ub
                search_alpha = (search_alpha+alpha_ub)/2
                _, termination_condition, halt_condition, summary_df = evaluate_alpha(model, search_alpha, current_gene, flux_df_col, target_obj_val, opt_df, alpha_lb , alpha_ub, exp_leap, precision)
        if(summary_df['Net_Flux'][0]>0 and 
           summary_df['div_opt_obj_val'][0]>0):
            opt_df = summary_df
#     while (np.floor(opt_df['div_opt_alpha'][0]*10**(precision-3))/10**(precision-3)-1 == 0):
#         precision += 1
#         print(precisio)
    print(current_gene, opt_df['div_opt_alpha'][0], opt_df[f'div_opt_obj_val'][0])
#     opt_df['div_opt_alpha'] = np.floor(opt_df['div_opt_alpha'][0]*10**(precision))/10**(precision)
    
    opt_df.columns = [f'{flux_df_col}_'+element for element in opt_df.columns]
    return(opt_df[f'{flux_df_col}_div_opt_alpha'],opt_df[f'{flux_df_col}_div_opt_obj_val'],opt_df) # alpha_feasible, and upper bound of alpha
# aa, _ = combine_div_obj_df(alpha_df = alpha_df)
# aa

workflow for generating objective value


In [463]:
%store list_target_obj_val

Stored 'list_target_obj_val' (list)


In [462]:
list_target_obj_val = [E0.slim_optimize(),
                       S0_ac.slim_optimize(),
                       S0_glc.slim_optimize()]
# list_target_obj_val[0]

In [464]:
import gc
gc.collect()

199865

In [116]:
list_target_obj_val = pd.DataFrame([E0.slim_optimize(),S0_ac.slim_optimize(),S0_glc.slim_optimize()]).T
list_target_obj_val.columns = ['E0','S0_ac','S0_glc']
def add_prefix_div_df(df, flux_df_col): # add prefix corresponding to given flux_df_col
    df.columns = [f'{flux_df_col}_{col}' for col in df.columns]
    
def get_div_obj_df(model, flux_df_col,target_obj_val,  
#                    first_n_gene=3, alpha_df = None): 
                   first_n_gene=len(obj_flux_df.index[1:]), alpha_df = None): 
    # generate full biomass dataframe for a single species
    # given alpha or automatic search for optimal with the given objective value
    with model:
        obj_div_df = pd.DataFrame()
        for i, current_gene in enumerate(obj_flux_df.index[1:][:first_n_gene]): # iter gene 
#         for i, current_gene in enumerate(obj_flux_df.index[1:][:1]): # iter gene 
            print(current_gene,'in cal')

            if alpha_df is None:
                if(current_gene=='yrbG'):
                    alpha,obj_value,temp_df = Sij_obj(model, alphas = 1.00001,
                                                      genes= 'yrbG', flux_df_col = flux_df_col)
                    temp_df.columns = [f'{flux_df_col}_'+element for element in temp_df.columns]
                else:
                    alpha,obj_value,temp_df = find_feasible_alpha(model, 1.02, current_gene, flux_df_col, 
                                                              target_obj_val, precision=8)
            else:
                alpha = alpha_df.loc[current_gene,f'{flux_df_col}_div_opt_alpha'] # get alpha for corresponding gene from alpha_df
                alpha,obj_value,temp_df = Sij_obj(model, alpha, #.pop?
                                              genes = current_gene, flux_df_col = flux_df_col)

            temp_df['Gene_inhibition'] = current_gene    
    #         print(alpha, current_gene, )
            obj_div_df = pd.concat([obj_div_df, temp_df.set_index('Gene_inhibition')],axis=0)
        add_prefix_div_df(obj_div_df, f'{flux_df_col}') 
#         print(obj_div_df.index)
    return(obj_div_df)
def combine_div_obj_df(list_target_obj_val=list_target_obj_val, alpha_df = None):
    # join obj_dfs of three cultures
    # return a full dataframe and list containing the three separate cultures
    
    obj_div_df = pd.DataFrame()
    # obj_div_df = pd.DataFrame(obj_flux_df.index[1:])
    E0_df = get_div_obj_df(E0, 'E0', target_obj_val=list_target_obj_val['E0'][0], 
                           alpha_df=alpha_df)
    S0_ac_df = get_div_obj_df(S0_ac, 'S0_ac', target_obj_val=list_target_obj_val['S0_ac'][0],
                              alpha_df=alpha_df)
    S0_glc_df = get_div_obj_df(S0_glc, 'S0_glc', target_obj_val=list_target_obj_val['S0_glc'][0],
                               alpha_df=alpha_df)

    obj_div_df = E0_df.join(S0_ac_df)
    obj_div_df = obj_div_df.join(S0_glc_df)
    
    pathway_list = list() # add pathway column
    for _, current_gene in enumerate(obj_div_df.index):
        pathway_sub = list()
        for _, rct in enumerate(E0.genes.get_by_id(get_gene_id(E0, current_gene)).reactions):
            if rct.id in set(S2tab['Enzyme']):
                pathway_sub.extend([S2tab.query(f"Enzyme=='{rct.id}'")['Pathway'].to_string(index = False)])
        pathway_list.append(', '.join(ele for ele in pathway_sub))

    classify_gene = list() # add gene_source
    for current_gene in obj_div_df.index:
        if current_gene in essential_genes:
            classify_gene.append('Drug Similarity')
        else:
            classify_gene.append('div S2table')
    
    obj_div_df = pd.concat([pd.DataFrame(pathway_list, columns=['Pathway'], index=obj_div_df.index),
                            pd.DataFrame(classify_gene, columns=['Gene_Source'], index=obj_div_df.index),
                            obj_div_df], axis=1)
#     obj_div_df.insert(0,'Gene_Source', classify_gene)
#     obj_div_df.insert(0,"Pathway", pathway_list)
        
    return(obj_div_df, [E0_df, S0_ac_df, S0_glc_df])
def rearrange_rows(df,filename): # row order accoding to gene source and E0 biomass
    df = df.set_index('Gene_inhibition')
    df = df.sort_values(by = ['Gene_Source', 'E0_div_opt_obj_val'])
#     df = df.sort_values(by = ['Gene_Source', 'div_opt_obj_val_x'])
    df.to_csv(f'{filename}.csv')
    return(df)

## Implementations

In [434]:
%store obj_div_df_90percent

Stored 'obj_div_df_90percent' (DataFrame)


In [168]:
# # obj_div_df_1percent, model_df_1percent = combine_div_obj_df(list_target_obj_val/100)
# obj_div_df_50percent, model_df_50percent = combine_div_obj_df(list_target_obj_val/2)
obj_div_df_90percent, model_df_90percent = combine_div_obj_df(list_target_obj_val*0.9)

In [468]:
obj_div_df_50percent.columns,obj_div_df_90percent.columns

(Index(['Pathway', 'Gene_Source', 'E0_div_opt_alpha', 'E0_div_opt_obj_val',
        'E0_FVAdir_Reactions_involved', 'E0_FVAdir_Flux_values',
        'E0_FVAopposite_Reactions_involved', 'E0_FVAopposite_Flux_values',
        'E0_Net_Flux_Boolean', 'E0_Net_Flux', 'S0_ac_div_opt_alpha',
        'S0_ac_div_opt_obj_val', 'S0_ac_FVAdir_Reactions_involved',
        'S0_ac_FVAdir_Flux_values', 'S0_ac_FVAopposite_Reactions_involved',
        'S0_ac_FVAopposite_Flux_values', 'S0_ac_Net_Flux_Boolean',
        'S0_ac_Net_Flux', 'S0_glc_div_opt_alpha', 'S0_glc_div_opt_obj_val',
        'S0_glc_FVAdir_Reactions_involved', 'S0_glc_FVAdir_Flux_values',
        'S0_glc_FVAopposite_Reactions_involved',
        'S0_glc_FVAopposite_Flux_values', 'S0_glc_Net_Flux_Boolean',
        'S0_glc_Net_Flux'],
       dtype='object'),
 Index(['Pathway', 'Gene_Source', 'E0_E0_div_opt_alpha',
        'E0_E0_div_opt_obj_val', 'E0_E0_FVAdir_Reactions_id',
        'E0_E0_FVAdir_Reactions_involved', 'E0_E0_FVAdir_Flux_valu

In [129]:
float(obj_div_df_90percent[['E0_E0_div_opt_alpha','S0_ac_S0_ac_div_opt_alpha', 'S0_glc_S0_glc_div_opt_alpha']])

TypeError: float() argument must be a string or a real number, not 'DataFrame'

In [ ]:
obj_div_df_90percent[['E0_E0_div_opt_alpha','S0_ac_S0_ac_div_opt_alpha', 'S0_glc_S0_glc_div_opt_alpha']]

In [159]:
compare_5090 = obj_div_df_50percent.join(
    obj_div_df_90percent[['E0_E0_div_opt_alpha',
                          'S0_ac_S0_ac_div_opt_alpha', 
                          'S0_glc_S0_glc_div_opt_alpha']].astype(float))

In [178]:
# compare_5090= pd.DataFrame()
alpha_range = pd.DataFrame()
compare_5090['keep'] = False
for col in ['E0_','S0_ac_', 'S0_glc_']:
    alpha_range[f'{col}max'] = compare_5090[f'{col}div_opt_alpha']
    alpha_range[f'{col}min'] = compare_5090[f'{col}{col}div_opt_alpha']
    
    compare_5090[f'{col}diff'] = compare_5090[f'{col}div_opt_alpha']-compare_5090[f'{col}{col}div_opt_alpha']
    compare_5090[f'{col}keep'] = compare_5090[f'{col}diff']>1e-5
    compare_5090['keep'] = compare_5090['keep'] | compare_5090[f'{col}keep']
alpha_range = alpha_range[compare_5090['keep']==True]
qualified_genes = list(alpha_range.index)
# qualified_genes

## aux

In [ ]:
# %store obj_div_df_1percent
# %store obj_div_df_50percent
# %store obj_div_df_90percent

# obj_div_df_1p.to_csv('obj_div_1p.csv')
# obj_div_df_50p.to_csv('obj_div_50p.csv')
# obj_div_df_90p.to_csv('obj_div_90p.csv')

_ = rearrange_rows(obj_div_df_1percent, 'obj_div_1p')


In [ ]:
obj_div_df_90percent = pd.read_csv('opt_flux_full_90p.csv')
obj_div_df_90percent.columns
obj_div_df_90percent = pd.concat([obj_div_df_90percent, df],axis=0)
%store obj_div_df_90percent
obj_div_df_90percent.to_csv('opt_flux_full_90p.csv')

In [ ]:
df = pd.DataFrame(columns = obj_div_df_1percent.columns, index=pd.Index(['Normal']))

for _, col in enumerate(df.columns):
    if 'div_opt_obj_val' in col:      
        species =  col.replace('_div_opt_obj_val','')        
        df.loc[df.index =='Normal', col] = list_target_obj_val[species][0]
        print(df.loc[df.index =='Normal', col])
#         df.loc = list_target_obj_val[col.split('_')[0]]

# Gene pairs inhibition

In [117]:
def add_synergy_label(df): # dominant, synergistic, oppression 
    eval_df = pd.DataFrame()
    eval_df['max'] = df[["First_gene_inhibition_50p_obj_val", "Second_gene_inhibition_50p_obj_val"]].max(axis=1)
    eval_df['min'] = df[["First_gene_inhibition_50p_obj_val", "Second_gene_inhibition_50p_obj_val"]].min(axis=1)

    synergy_cond = eval_df['min'].round(5) > df["div_opt_obj_val"].round(5) # Synergy
    anatagonism_cond = ((eval_df['max'].round(5) >= df["div_opt_obj_val"].round(5)) & 
                        (eval_df['min'].round(5) < df["div_opt_obj_val"].round(5)))
    dominance_cond = (eval_df['min'].round(5) == df["div_opt_obj_val"].round(5))

    eval_df['Response'] = np.where(synergy_cond, 'Synergy', 
                                   np.where(anatagonism_cond, 'Anatagonism',
                                           np.where(dominance_cond, 'Dominance','Other')
                                           )
                                  )
    df.insert(0,'Drug_Response',eval_df['Response'])

# add_synergy_label(S0_ac_div_DG)
# S0_ac_div_DG.to_csv('ac_div.csv')

# Adjusted alpha single gene inihibitions

In [118]:
%store -r obj_div_df_50percent
obj_div_df_50percent = obj_div_df_50percent.set_index('Gene_inhibition')

In [23]:
Sij_obj(alphas=[1.4], genes=('gdhA'))

GLUDy duplicated reaction


(1.4,
 0.7541530691167869,
   div_opt_alpha  div_opt_obj_val FVAdir_Reactions_id  \
 0         [1.4]         0.754153                       
 
   FVAdir_Reactions_involved FVAdir_Flux_values FVAopposite_Reactions_involved  \
 0                                           []                                  
 
   FVAopposite_Flux_values Net_Flux_Boolean  Net_Flux  
 0                      []        Zero Flux         0  )

In [180]:
alpha_range


,E0_max,E0_min,S0_ac_max,S0_ac_min,S0_glc_max,S0_glc_min
Gene_inhibition,,,,,,
gapA,1.040,1.000,1.040,1.040,1.040,1.000
gltA,3.080,1.540,1.000,1.000,3.580,1.540
dapA,43.780,9.820,37.780,4.660,63.780,8.820
murA,191.620,26.140,54.780,6.660,107.060,12.820
talB,1.040,1.020,1.040,1.040,1.020,1.002
folP,3955.420,675.980,82.060,9.820,66.060,8.820
dadX,381.740,132.620,299.740,33.780,241.620,83.060
thrC,30.140,6.660,41.780,5.660,49.780,6.660
rffG,1.002,1.002,112.060,13.820,253.620,31.140


In [119]:
minimum_NG_alpha = 500
def create_alpha_df(obj_div_df, minimum_NG_alpha = minimum_NG_alpha):
#     alpha_df = obj_div_df[['E0_div_opt_alpha','S0_ac_div_opt_alpha', 'S0_glc_div_opt_alpha',
#                           'E0_Net_Flux','S0_ac_Net_Flux', 'S0_glc_Net_Flux']].copy()
    alpha_df = obj_div_df[['E0_div_opt_alpha','S0_ac_div_opt_alpha', 'S0_glc_div_opt_alpha']].copy()
         
    for col in ['E0','S0_ac','S0_glc']:
#         print(alpha_df[f'{col}_div_opt_alpha'], f'{col}_div_opt_alpha')
        alpha_df.loc[alpha_df[f'{col}_Net_Flux']==0,f'{col}_div_opt_alpha'] = minimum_NG_alpha # Correct for NG  ?Netflux col  
        alpha_df.loc[alpha_df[f'{col}_div_opt_alpha']>500, f'{col}_div_opt_alpha'] = 1000
        alpha_df.loc[alpha_df.index=='tktA',f'{col}_div_opt_alpha'] = 1.2 # minimum alpha without having reverse reactions

    if('Normal' in obj_div_df.index):
        alpha_df = alpha_df[:-1]
    return(alpha_df)

def get_scaled_alphas(alphas, alpha_scale):
    convert_arg_to_list(alphas)
    return(list(1+np.multiply([ele-1 for ele in alphas], alpha_scale)))

# alpha_df = create_alpha_df(obj_div_df_50percent)
# scaled_alpha_dict = get_scaled_alpha_df(alpha_df)

def get_alpha_intervals(alpha_range, col):
    return(pd.DataFrame(alpha_range.apply(lambda x:
                          np.linspace(x[f'{col}_min'],x[f'{col}_max'],5).round(3)
                                          if x[f'{col}_max']-x[f'{col}_min'] > 1e-5 else None,
                          axis=1),
                       columns=['alpha_interval']))

def concat_species_df_col_only(f,*args,**kwargs): 
    df = pd.DataFrame()
    for col in ['E0', 'S0_ac', 'S0_glc']:
        df_temp= f(*args,col,**kwargs)
        df = pd.concat([df, df_temp.add_prefix(f'{col}_')], axis = 1)
    return(df)

# def concat_species_df(f,*args,**kwargs): 
#     df = pd.DataFrame()
#     for model, col in zip([E0, S0_ac, S0_glc], ['E0','S0_ac','S0_glc']):
#         df_temp= f(*args,model,col,**kwargs)
#         df = pd.concat([df, df_temp.add_prefix(f'{col}_')], axis = 1)
#     return(df)

In [25]:
# zipped= zip([E0, S0_ac, S0_glc], ['E0','S0_ac','S0_glc'])
zipped= zip(['E0','S0_ac','S0_glc'])

for *model,col in zipped:
    print(type(model))
    print(model)

<class 'list'>
[]
<class 'list'>
[]
<class 'list'>
[]


In [67]:
from inspect import Parameter, Signature
fields = ['name', 'shares', 'price']
parms = [ Parameter(name,
 Parameter.POSITIONAL_OR_KEYWORD)
 for name in fields]
sig = Signature(parms)
parms

[<Parameter "name">, <Parameter "shares">, <Parameter "price">]

In [76]:
from inspect import Parameter, Signature
def make_signature(names):
    return Signature(
        Parameter(name,
            Parameter.POSITIONAL_OR_KEYWORD)
        for name in names)
class Structure:
    __signature__ = make_signature([])
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(
            *args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

class Stock(Structure):
     __signature__ = make_signature(
     ['name','shares','price'])

a = 'pos'
aa = Stock(a,a,a)

In [80]:
import inspect
print(inspect.signature(Stock))

(name, shares, price)


In [77]:
s = Stock('ACME', shares=50, price=91.1)

In [61]:
os.environ['DEBUG_VALUE'] = 'None'
del os.environ['DEBUG_VALUE'] 
from functools import wraps
def debug(func):
    if 'DEBUG_VALUE' not in os.environ:
         return func
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

@debug
def add(x,y):
    return x+y

add(1,2)

3

In [15]:
def concat_species_df_col_only(f,zipped,*args,**kwargs): 
    df = pd.DataFrame()
    for *model,col in zipped:
        if model == list():
            df_temp= f(*args,col,**kwargs)
        else: 
            df_temp= f(*args,model, col,**kwargs)
        df = pd.concat([df, df_temp.add_prefix(f'{col}_')], axis = 1)
    return(df)



In [16]:
concat_species_df_col_only(, ['E0','S0_ac'],alpha_range)

NameError: name 'alpha_range' is not defined

In [314]:
alpha_intervals_df = concat_species_df_col_only(get_alpha_intervals, alpha_range)

In [307]:
alpha_intervals_df.loc['mrdA']

E0_alpha_interval        [15.82, 29.88, 43.94, 58.0, 72.06]
S0_ac_alpha_interval                                   None
S0_glc_alpha_interval                                  None
Name: mrdA, dtype: object

In [192]:
# np.linspace(alpha_range['E0_max'],alpha_range['E0_min'])

In [ ]:
# no use?
for model, flux_df_col in zip([E0, S0_ac, S0_glc], ['E0','S0_ac','S0_glc']):
    for scale_alpha in scale_alpha_list: 
        alphas = (float(scaled_alpha_dict[scale_alpha[0]] # first scale_alpha df from scaled_alpha_dict
                  .at[gene_pair[0],f'{flux_df_col}_div_opt_alpha']), # alpha corresponding to first gene and first scale
                  float(scaled_alpha_dict[scale_alpha[1]]
                  .at[gene_pair[1],f'{flux_df_col}_div_opt_alpha'])) # alpha corresponding to second gene and second scale
        obj_val = Sij_obj(model,alphas, gene_pair,flux_df_col,slim_opt=True)
    #             print(gene_pair,alphas,obj_val)
        temp_dict[flux_df_col].append(obj_val)

# New pair_wise dict


In [451]:
len(alpha_intervals_df.index)

32

In [344]:
alpha_levels = [(i,j) for i in reversed(np.arange(5)) for j in np.arange(5)] 
alpha_level_comb = alpha_levels[0]
(alpha_intervals_df.at[gene_pair[0],f'{flux_df_col}_alpha_interval'][alpha_level_comb[0]])


1.04

In [467]:
len(alpha_intervals_df.index)

32

In [452]:

from collections import defaultdict
from time import time
alpha_levels = [(i,j) for i in reversed(np.arange(5)) for j in np.arange(5)] 
pw_dict_index = alpha_levels.copy()
pw_dict_index.extend(['y_ticks','x_ticks'])
pw_drug_response_dict = defaultdict(pd.DataFrame) # pair_wise

before= time()
# for gene_pair in itertools.combinations(('mrdA','folA'),2): # scaling for NG undertermined 
# for gene_pair in itertools.combinations(list(genes_dict['higher_alpha'])[:],2): # scaling for NG undertermined     
for gene_pair in itertools.combinations(alpha_intervals_df.index,2): # scaling for NG undertermined     
    temp_dict = defaultdict(list)
    print(gene_pair)
    for model, col in zip([E0, S0_ac, S0_glc], ['E0','S0_ac','S0_glc']):
        alpha_intervals = (alpha_intervals_dict[f'{col}_alpha_interval'][gene_pair[0]],
                          alpha_intervals_dict[f'{col}_alpha_interval'][gene_pair[1]])
        print(alpha_intervals)
#         alpha_intervals = (alpha_intervals_df.at[gene_pair[0], f'{col}_alpha_interval'],
#                            alpha_intervals_df.at[gene_pair[1], f'{col}_alpha_interval'])
        if all(ele is not None for ele in alpha_intervals): # alpha max, min != constant
            for alpha_level_comb in alpha_levels:
                alphas = (alpha_intervals[0][alpha_level_comb[0]],
                          alpha_intervals[1][alpha_level_comb[1]])
                obj_val = Sij_obj(model,alphas, gene_pair,col,slim_opt=True)
    #             print(gene_pair,alphas,obj_val)
                temp_dict[col].append(obj_val)
            temp_dict[col].append(alpha_intervals[0])
            temp_dict[col].append(alpha_intervals[1])
        else: temp_dict[col] = None
        
    pw_drug_response_dict[gene_pair] = pd.DataFrame(temp_dict,index=pw_dict_index)
    
#     find_gene_pair_inhibition(obj_div_df_50percent, 
#                                    genes_dict, minimum_NG_alpha = 1.02, scale_alpha=[1,3])
after = time()
print(after-before)

%store pw_drug_response_dict

('gapA', 'gltA')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([1.54 , 1.925, 2.31 , 2.695, 3.08 ]))
(None, None)
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([1.54, 2.05, 2.56, 3.07, 3.58]))
('gapA', 'dapA')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 9.82, 18.31, 26.8 , 35.29, 43.78]))
(None, array([ 4.66, 12.94, 21.22, 29.5 , 37.78]))
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 8.82, 22.56, 36.3 , 50.04, 63.78]))
('gapA', 'murA')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 26.14,  67.51, 108.88, 150.25, 191.62]))
(None, array([ 6.66, 18.69, 30.72, 42.75, 54.78]))
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 12.82,  36.38,  59.94,  83.5 , 107.06]))
('gapA', 'talB')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([1.02 , 1.025, 1.03 , 1.035, 1.04 ]))
(None, None)
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([1.002, 1.006, 1.011, 1.016, 1.02 ]))
('gapA', 'folP')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 675.98, 1495.84, 2315.7 , 3135.56, 3955.42]))
(None, array([ 9

('gltA', 'aroA')
(array([1.54 , 1.925, 2.31 , 2.695, 3.08 ]), array([ 1.54,  9.85, 18.16, 26.47, 34.78]))
(None, array([ 2.58,  6.72, 10.86, 15.  , 19.14]))
(array([1.54, 2.05, 2.56, 3.07, 3.58]), array([ 5.66, 13.69, 21.72, 29.75, 37.78]))
('gltA', 'ackA')
(array([1.54 , 1.925, 2.31 , 2.695, 3.08 ]), array([1.  , 1.01, 1.02, 1.03, 1.04]))
(None, array([1.   , 1.005, 1.01 , 1.015, 1.02 ]))
(array([1.54, 2.05, 2.56, 3.07, 3.58]), None)
('gltA', 'serC')
(array([1.54 , 1.925, 2.31 , 2.695, 3.08 ]), array([ 4.66,  9.03, 13.4 , 17.77, 22.14]))
(None, None)
(array([1.54, 2.05, 2.56, 3.07, 3.58]), None)
('gltA', 'pyrD')
(array([1.54 , 1.925, 2.31 , 2.695, 3.08 ]), None)
(None, array([1.04 , 1.425, 1.81 , 2.195, 2.58 ]))
(array([1.54, 2.05, 2.56, 3.07, 3.58]), array([1.54, 2.32, 3.1 , 3.88, 4.66]))
('gltA', 'acnB')
(array([1.54 , 1.925, 2.31 , 2.695, 3.08 ]), array([2.58, 3.35, 4.12, 4.89, 5.66]))
(None, None)
(array([1.54, 2.05, 2.56, 3.07, 3.58]), array([2.58, 3.35, 4.12, 4.89, 5.66]))
('glt

('dapA', 'folA')
(array([ 9.82, 18.31, 26.8 , 35.29, 43.78]), array([ 8.82, 17.31, 25.8 , 34.29, 42.78]))
(array([ 4.66, 12.94, 21.22, 29.5 , 37.78]), array([ 3.58,  7.72, 11.86, 16.  , 20.14]))
(array([ 8.82, 22.56, 36.3 , 50.04, 63.78]), array([ 3.58,  7.97, 12.36, 16.75, 21.14]))
('dapA', 'thrB')
(array([ 9.82, 18.31, 26.8 , 35.29, 43.78]), array([1.58, 2.6 , 3.62, 4.64, 5.66]))
(array([ 4.66, 12.94, 21.22, 29.5 , 37.78]), array([ 1.58,  3.89,  6.2 ,  8.51, 10.82]))
(array([ 8.82, 22.56, 36.3 , 50.04, 63.78]), array([2.58, 4.39, 6.2 , 8.01, 9.82]))
('dapA', 'folC')
(array([ 9.82, 18.31, 26.8 , 35.29, 43.78]), array([ 383.74,  782.42, 1181.1 , 1579.78, 1978.46]))
(array([ 4.66, 12.94, 21.22, 29.5 , 37.78]), array([ 5.66, 15.69, 25.72, 35.75, 45.78]))
(array([ 8.82, 22.56, 36.3 , 50.04, 63.78]), array([ 5.66, 13.94, 22.22, 30.5 , 38.78]))
('dapA', 'dapF')
(array([ 9.82, 18.31, 26.8 , 35.29, 43.78]), array([ 7.82, 11.4 , 14.98, 18.56, 22.14]))
(array([ 4.66, 12.94, 21.22, 29.5 , 37.78]

('murA', 'dapF')
(array([ 26.14,  67.51, 108.88, 150.25, 191.62]), array([ 7.82, 11.4 , 14.98, 18.56, 22.14]))
(array([ 6.66, 18.69, 30.72, 42.75, 54.78]), array([ 4.66, 11.53, 18.4 , 25.27, 32.14]))
(array([ 12.82,  36.38,  59.94,  83.5 , 107.06]), array([ 6.66, 13.44, 20.22, 27.  , 33.78]))
('murA', 'guaB')
(array([ 26.14,  67.51, 108.88, 150.25, 191.62]), None)
(array([ 6.66, 18.69, 30.72, 42.75, 54.78]), array([1.54, 3.07, 4.6 , 6.13, 7.66]))
(array([ 12.82,  36.38,  59.94,  83.5 , 107.06]), array([1.54, 3.36, 5.18, 7.  , 8.82]))
('murA', 'pyrE')
(array([ 26.14,  67.51, 108.88, 150.25, 191.62]), array([ 4.66,  9.78, 14.9 , 20.02, 25.14]))
(array([ 6.66, 18.69, 30.72, 42.75, 54.78]), array([ 2.58,  6.72, 10.86, 15.  , 19.14]))
(array([ 12.82,  36.38,  59.94,  83.5 , 107.06]), array([ 3.58,  9.47, 15.36, 21.25, 27.14]))
('murA', 'dapB')
(array([ 26.14,  67.51, 108.88, 150.25, 191.62]), array([1.54, 2.82, 4.1 , 5.38, 6.66]))
(array([ 6.66, 18.69, 30.72, 42.75, 54.78]), array([1.54, 2.

('talB', 'dapD')
(array([1.02 , 1.025, 1.03 , 1.035, 1.04 ]), array([1.54, 2.82, 4.1 , 5.38, 6.66]))
(None, array([1.54, 2.57, 3.6 , 4.63, 5.66]))
(array([1.002, 1.006, 1.011, 1.016, 1.02 ]), array([1.54, 3.07, 4.6 , 6.13, 7.66]))
('talB', 'pheA')
(array([1.02 , 1.025, 1.03 , 1.035, 1.04 ]), array([1.54, 3.36, 5.18, 7.  , 8.82]))
(None, array([2.58, 3.6 , 4.62, 5.64, 6.66]))
(array([1.002, 1.006, 1.011, 1.016, 1.02 ]), array([3.58, 4.89, 6.2 , 7.51, 8.82]))
('talB', 'argD')
(array([1.02 , 1.025, 1.03 , 1.035, 1.04 ]), array([ 3.58,  5.89,  8.2 , 10.51, 12.82]))
(None, array([ 2.58,  5.64,  8.7 , 11.76, 14.82]))
(array([1.002, 1.006, 1.011, 1.016, 1.02 ]), array([ 3.58,  7.72, 11.86, 16.  , 20.14]))
('talB', 'trpA')
(array([1.02 , 1.025, 1.03 , 1.035, 1.04 ]), array([ 20.14,  53.76,  87.38, 121.  , 154.62]))
(None, array([ 4.66, 12.94, 21.22, 29.5 , 37.78]))
(array([1.002, 1.006, 1.011, 1.016, 1.02 ]), array([ 9.82, 25.63, 41.44, 57.25, 73.06]))
('talB', 'gltD')
(array([1.02 , 1.025, 1.

...

('acnB', 'gltD')
(array([2.58, 3.35, 4.12, 4.89, 5.66]), array([1.03 , 1.032, 1.035, 1.038, 1.04 ]))
(None, None)
(array([2.58, 3.35, 4.12, 4.89, 5.66]), None)
('pgk', 'eno')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([1.04 , 1.945, 2.85 , 3.755, 4.66 ]))
(None, None)
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([1.04 , 1.675, 2.31 , 2.945, 3.58 ]))
('pgk', 'folA')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 8.82, 17.31, 25.8 , 34.29, 42.78]))
(None, array([ 3.58,  7.72, 11.86, 16.  , 20.14]))
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 3.58,  7.97, 12.36, 16.75, 21.14]))
('pgk', 'thrB')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([1.58, 2.6 , 3.62, 4.64, 5.66]))
(None, array([ 1.58,  3.89,  6.2 ,  8.51, 10.82]))
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([2.58, 4.39, 6.2 , 8.01, 9.82]))
('pgk', 'folC')
(array([1.  , 1.01, 1.02, 1.03, 1.04]), array([ 383.74,  782.42, 1181.1 , 1579.78, 1978.46]))
(None, array([ 5.66, 15.69, 25.72, 35.75, 45.78]))
(array([1.  , 1.01, 1.0

(array([ 3.58,  7.97, 12.36, 16.75, 21.14]), array([1.54, 3.36, 5.18, 7.  , 8.82]))
('folA', 'pyrE')
(array([ 8.82, 17.31, 25.8 , 34.29, 42.78]), array([ 4.66,  9.78, 14.9 , 20.02, 25.14]))
(array([ 3.58,  7.72, 11.86, 16.  , 20.14]), array([ 2.58,  6.72, 10.86, 15.  , 19.14]))
(array([ 3.58,  7.97, 12.36, 16.75, 21.14]), array([ 3.58,  9.47, 15.36, 21.25, 27.14]))
('folA', 'dapB')
(array([ 8.82, 17.31, 25.8 , 34.29, 42.78]), array([1.54, 2.82, 4.1 , 5.38, 6.66]))
(array([ 3.58,  7.72, 11.86, 16.  , 20.14]), array([1.54, 2.82, 4.1 , 5.38, 6.66]))
(array([ 3.58,  7.97, 12.36, 16.75, 21.14]), array([1.54, 3.36, 5.18, 7.  , 8.82]))
('folA', 'purT')
(array([ 8.82, 17.31, 25.8 , 34.29, 42.78]), array([1.  , 1.01, 1.02, 1.03, 1.04]))
(array([ 3.58,  7.72, 11.86, 16.  , 20.14]), array([1.   , 1.005, 1.01 , 1.015, 1.02 ]))
(array([ 3.58,  7.97, 12.36, 16.75, 21.14]), None)
('folA', 'mrdA')
(array([ 8.82, 17.31, 25.8 , 34.29, 42.78]), array([15.82, 29.88, 43.94, 58.  , 72.06]))
(array([ 3.58,  

('folC', 'pfkA')
(array([ 383.74,  782.42, 1181.1 , 1579.78, 1978.46]), None)
(array([ 5.66, 15.69, 25.72, 35.75, 45.78]), None)
(array([ 5.66, 13.94, 22.22, 30.5 , 38.78]), array([1.03 , 1.032, 1.035, 1.038, 1.04 ]))
('folC', 'dapD')
(array([ 383.74,  782.42, 1181.1 , 1579.78, 1978.46]), array([1.54, 2.82, 4.1 , 5.38, 6.66]))
(array([ 5.66, 15.69, 25.72, 35.75, 45.78]), array([1.54, 2.57, 3.6 , 4.63, 5.66]))
(array([ 5.66, 13.94, 22.22, 30.5 , 38.78]), array([1.54, 3.07, 4.6 , 6.13, 7.66]))
('folC', 'pheA')
(array([ 383.74,  782.42, 1181.1 , 1579.78, 1978.46]), array([1.54, 3.36, 5.18, 7.  , 8.82]))
(array([ 5.66, 15.69, 25.72, 35.75, 45.78]), array([2.58, 3.6 , 4.62, 5.64, 6.66]))
(array([ 5.66, 13.94, 22.22, 30.5 , 38.78]), array([3.58, 4.89, 6.2 , 7.51, 8.82]))
('folC', 'argD')
(array([ 383.74,  782.42, 1181.1 , 1579.78, 1978.46]), array([ 3.58,  5.89,  8.2 , 10.51, 12.82]))
(array([ 5.66, 15.69, 25.72, 35.75, 45.78]), array([ 2.58,  5.64,  8.7 , 11.76, 14.82]))
(array([ 5.66, 13.9

(array([ 2.58,  6.72, 10.86, 15.  , 19.14]), array([1.54, 2.57, 3.6 , 4.63, 5.66]))
(array([ 3.58,  9.47, 15.36, 21.25, 27.14]), array([1.54, 3.07, 4.6 , 6.13, 7.66]))
('pyrE', 'pheA')
(array([ 4.66,  9.78, 14.9 , 20.02, 25.14]), array([1.54, 3.36, 5.18, 7.  , 8.82]))
(array([ 2.58,  6.72, 10.86, 15.  , 19.14]), array([2.58, 3.6 , 4.62, 5.64, 6.66]))
(array([ 3.58,  9.47, 15.36, 21.25, 27.14]), array([3.58, 4.89, 6.2 , 7.51, 8.82]))
('pyrE', 'argD')
(array([ 4.66,  9.78, 14.9 , 20.02, 25.14]), array([ 3.58,  5.89,  8.2 , 10.51, 12.82]))
(array([ 2.58,  6.72, 10.86, 15.  , 19.14]), array([ 2.58,  5.64,  8.7 , 11.76, 14.82]))
(array([ 3.58,  9.47, 15.36, 21.25, 27.14]), array([ 3.58,  7.72, 11.86, 16.  , 20.14]))
('pyrE', 'trpA')
(array([ 4.66,  9.78, 14.9 , 20.02, 25.14]), array([ 20.14,  53.76,  87.38, 121.  , 154.62]))
(array([ 2.58,  6.72, 10.86, 15.  , 19.14]), array([ 4.66, 12.94, 21.22, 29.5 , 37.78]))
(array([ 3.58,  9.47, 15.36, 21.25, 27.14]), array([ 9.82, 25.63, 41.44, 57.25,

('pfkA', 'trpA')
(None, array([ 20.14,  53.76,  87.38, 121.  , 154.62]))
(None, array([ 4.66, 12.94, 21.22, 29.5 , 37.78]))
(array([1.03 , 1.032, 1.035, 1.038, 1.04 ]), array([ 9.82, 25.63, 41.44, 57.25, 73.06]))
('pfkA', 'gltD')
(None, array([1.03 , 1.032, 1.035, 1.038, 1.04 ]))
(None, None)
(array([1.03 , 1.032, 1.035, 1.038, 1.04 ]), None)
('dapD', 'pheA')
(array([1.54, 2.82, 4.1 , 5.38, 6.66]), array([1.54, 3.36, 5.18, 7.  , 8.82]))
(array([1.54, 2.57, 3.6 , 4.63, 5.66]), array([2.58, 3.6 , 4.62, 5.64, 6.66]))
(array([1.54, 3.07, 4.6 , 6.13, 7.66]), array([3.58, 4.89, 6.2 , 7.51, 8.82]))
('dapD', 'argD')
(array([1.54, 2.82, 4.1 , 5.38, 6.66]), array([ 3.58,  5.89,  8.2 , 10.51, 12.82]))
(array([1.54, 2.57, 3.6 , 4.63, 5.66]), array([ 2.58,  5.64,  8.7 , 11.76, 14.82]))
(array([1.54, 3.07, 4.6 , 6.13, 7.66]), array([ 3.58,  7.72, 11.86, 16.  , 20.14]))
('dapD', 'trpA')
(array([1.54, 2.82, 4.1 , 5.38, 6.66]), array([ 20.14,  53.76,  87.38, 121.  , 154.62]))
(array([1.54, 2.57, 3.6 , 

In [456]:
pw_drug_response_dict.info

AttributeError: 'collections.defaultdict' object has no attribute 'info'


## Lower inhibition combinations

In [98]:
genes_dict = {'Potential_genes': potential_genes}
genes_dict['min_alpha'] =  set(alpha_df.index[alpha_df['E0_div_opt_alpha']<=1.002])
genes_dict['higher_alpha'] = potential_genes - genes_dict['min_alpha']


In [ ]:
test_list = [ele for ele in itertools.combinations(genes_dict['higher_alpha'], 2)]
len(test_list)

# pair-wise dict

In [105]:
alpha_df = create_alpha_df(obj_div_df_50percent,1.002)
genes_dict = {'Potential_genes': potential_genes}
genes_dict['min_alpha'] =  set(alpha_df.index[alpha_df['E0_div_opt_alpha']<=1.002])
genes_dict['higher_alpha'] = potential_genes - genes_dict['min_alpha']


In [209]:
scale_alpha = scale_alpha_list[0]
scale_alpha[0]

gene_pair =('folA','folP')
flux_df_col = 'E0'
scaled_alpha_dict[scale_alpha[0]].at[gene_pair[0],f'{flux_df_col}_div_opt_alpha']

['18.112000000000002']

In [242]:
# Sij_obj(genes= 'folP', alphas=200, slim_opt=True)
df = pd.DataFrame([[1.2, 2.2], [4.2, .5], [.7, .8]],
     index=['cobra', 'viper', 'sidewinder'],
     columns=['max_speed', 'shield'])
df.loc['cobra', 'shield']
df.dtypes

max_speed    float64
shield       float64
dtype: object

In [258]:
%store pw_drug_response_dict

Stored 'pw_drug_response_dict' (defaultdict)


In [86]:
from collections import defaultdict
from time import time
scale_alpha_list = [(i,j) for i in reversed(np.linspace(1/5,1,5).round(3)) 
                              for j in np.linspace(1/5,1,5).round(3)]
# scale_alpha_list = [(i,j) for i in reversed(np.linspace(1/5,2/5,2).round(3)) 
#                               for j in np.linspace(1/5,2/5,2).round(3)]
# scale_alpha_list = [(i,j) for i in reversed(np.linspace(0,1,5).round(3)) 
#                               for j in np.linspace(0,1,5).round(3)]
pw_drug_response_dict = defaultdict(pd.DataFrame) # pair_wise
# pw_drug_response_dict = defaultdict(list)

before= time()
for gene_pair in itertools.combinations(('purT', 'dapA'),2): # scaling for NG undertermined 
# for gene_pair in itertools.combinations(list(genes_dict['higher_alpha'])[:],2): # scaling for NG undertermined 
    temp_dict = defaultdict(list)
    print(gene_pair)
    for model, flux_df_col in zip([E0, S0_ac, S0_glc], ['E0','S0_ac','S0_glc']):
        for scale_alpha in scale_alpha_list: 
            alphas = (float(scaled_alpha_dict[scale_alpha[0]] # first scale_alpha df from scaled_alpha_dict
                      .at[gene_pair[0],f'{flux_df_col}_div_opt_alpha']), # alpha corresponding to first gene and first scale
                      float(scaled_alpha_dict[scale_alpha[1]]
                      .at[gene_pair[1],f'{flux_df_col}_div_opt_alpha'])) # alpha corresponding to second gene and second scale
            obj_val = Sij_obj(model,alphas, gene_pair,flux_df_col,slim_opt=True)
#             print(gene_pair,alphas,obj_val)
            temp_dict[flux_df_col].append(obj_val)
    
    pw_drug_response_dict[gene_pair] = pd.DataFrame(temp_dict,index=scale_alpha_list)
    
#     find_gene_pair_inhibition(obj_div_df_50percent, 
#                                    genes_dict, minimum_NG_alpha = 1.02, scale_alpha=[1,3])
after = time()
print(after-before)

%store pw_drug_response_dict

('purT', 'dapA')
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['ACKr', 'ACKr_v1']
['A

In [ ]:
minimum_NG_alpha = 500
def fill_div_df(model, alpha_df, gene_pairs, df, flux_df_col, obj_div_df, scale_alpha, slim_opt=False):
    alphas = [alpha_df[f'{flux_df_col}_div_opt_alpha'][gene_pairs[0]], 
              alpha_df[f'{flux_df_col}_div_opt_alpha'][gene_pairs[1]]]
#     print(model, alphas)
#     alphas = list(1+np.divide([ele-1 for ele in alphas], scale_alpha))
#     alphas = list(1+np.multiply([ele-1 for ele in alphas], scale_alpha))
    alphas = get_scaled_alphas(alphas, scale_alpha)
    if not slim_opt:
        _, obj_val, temp_df = Sij_obj(model,alphas, gene_pairs,flux_df_col,slim_opt=False)
        temp_df.insert(2,'Second_gene_pathway',
               obj_div_df['Pathway'][gene_pairs[1]])
        temp_df.insert(2,'First_gene_pathway',
               obj_div_df['Pathway'][gene_pairs[0]])

        temp_df.insert(2,'Second_gene_inhibition_50p_obj_val',
               obj_div_df[f'{flux_df_col}_div_opt_obj_val'][gene_pairs[1]])
        temp_df.insert(2,'Second_gene_inhibition',gene_pairs[1])
        temp_df.insert(2,'First_gene_inhibition_50p_obj_val',
                       obj_div_df[f'{flux_df_col}_div_opt_obj_val'][gene_pairs[0]])
        temp_df.insert(2,'First_gene_inhibition',gene_pairs[0])
        temp_df['Gene_inhibition'] = str(gene_pairs)
               
    temp_df = Sij_obj(model,alphas, gene_pairs,flux_df_col,slim_opt=True)
    df = pd.concat([df, temp_df],axis=0)
    return(df)


In [ ]:
def find_gene_pair_inhibition(alpha_df, gene_pairs,
                              minimum_NG_alpha=minimum_NG_alpha, scale_alpha=[1,1],slim_opt=False):
#     alpha_df = create_alpha_df(obj_div_df,minimum_NG_alpha)
    df_E0, df_S0_ac, df_S0_glc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame() # split
#     for gene_pairs in itertools.combinations(('gapA', 'dadX'), 2):
#     for gene_pairs in itertools.combinations(list(potential_genes)[:2], 2):  
#     if scale_alpha[0] ==1:
#         test_list = [(w,f) for w in genes_dict['Potential_genes'] for f in genes_dict['higher_alpha']]
#     else:
#         test_list = [ele for ele in itertools.combinations(genes_dict['higher_alpha'], 2)]
        
    for gene_pair in gene_pairs:        
        print(gene_pair)
        df_E0 = fill_div_df(E0, alpha_df,gene_pair, df_E0, 'E0',obj_div_df, scale_alpha,slim_opt)
        df_S0_ac = fill_div_df(S0_ac, alpha_df,gene_pair, df_S0_ac, 'S0_ac',obj_div_df, scale_alpha,slim_opt)
        df_S0_glc = fill_div_df(S0_glc, alpha_df,gene_pair, df_S0_glc, 'S0_glc',obj_div_df, scale_alpha,slim_opt)
    if not slim_opt:    
        df_E0 = df_E0.set_index('Gene_inhibition')
        df_S0_ac = df_S0_ac.set_index('Gene_inhibition')
        df_S0_glc = df_S0_glc.set_index('Gene_inhibition')    
        add_prefix_div_df(df_E0, 'E0')
        add_prefix_div_df(df_S0_ac, 'S0_ac')
        add_prefix_div_df(df_S0_glc,'S0_glc') 
    return([df_E0, df_S0_ac,df_S0_glc])


# prev version 

In [14]:
minimum_NG_alpha = 500

def fill_div_df(model, alpha_df, gene_pairs, df, flux_df_col, obj_div_df, scale_alpha, slim_opt=False):
    alphas = [alpha_df[f'{flux_df_col}_div_opt_alpha'][gene_pairs[0]], 
              alpha_df[f'{flux_df_col}_div_opt_alpha'][gene_pairs[1]]]
#     print(model, alphas)
#     alphas = list(1+np.divide([ele-1 for ele in alphas], scale_alpha))
#     alphas = list(1+np.multiply([ele-1 for ele in alphas], scale_alpha))
    alphas = get_scaled_alphas(alphas, scale_alpha)
    if not slim_opt:
        _, obj_val, temp_df = Sij_obj(model,alphas, gene_pairs,flux_df_col,slim_opt=False)
        temp_df.insert(2,'Second_gene_pathway',
               obj_div_df['Pathway'][gene_pairs[1]])
        temp_df.insert(2,'First_gene_pathway',
               obj_div_df['Pathway'][gene_pairs[0]])

        temp_df.insert(2,'Second_gene_inhibition_50p_obj_val',
               obj_div_df[f'{flux_df_col}_div_opt_obj_val'][gene_pairs[1]])
        temp_df.insert(2,'Second_gene_inhibition',gene_pairs[1])
        temp_df.insert(2,'First_gene_inhibition_50p_obj_val',
                       obj_div_df[f'{flux_df_col}_div_opt_obj_val'][gene_pairs[0]])
        temp_df.insert(2,'First_gene_inhibition',gene_pairs[0])
        temp_df['Gene_inhibition'] = str(gene_pairs)
               
        
    temp_df = Sij_obj(model,alphas, gene_pairs,flux_df_col,slim_opt=True)
    df = pd.concat([df, temp_df],axis=0)
    return(df)
def find_gene_pair_inhibition(obj_div_df, genes_dict,
                              minimum_NG_alpha=minimum_NG_alpha, scale_alpha=[1,1],slim_opt=False):
    alpha_df = create_alpha_df(obj_div_df,minimum_NG_alpha)
    df_E0, df_S0_ac, df_S0_glc = pd.DataFrame(),pd.DataFrame(),pd.DataFrame() # split
#     for gene_pairs in itertools.combinations(('gapA', 'dadX'), 2):
#     for gene_pairs in itertools.combinations(list(potential_genes)[:2], 2):  
    if scale_alpha[0] ==1:
        test_list = [(w,f) for w in genes_dict['Potential_genes'] for f in genes_dict['higher_alpha']]
    else:
        test_list = [ele for ele in itertools.combinations(genes_dict['higher_alpha'], 2)]
        
    for gene_pairs in test_list:        
        print(gene_pairs)
        df_E0 = fill_div_df(E0, alpha_df,gene_pairs, df_E0, 'E0',obj_div_df, scale_alpha,slim_opt)
        df_S0_ac = fill_div_df(S0_ac, alpha_df,gene_pairs, df_S0_ac, 'S0_ac',obj_div_df, scale_alpha,slim_opt)
        df_S0_glc = fill_div_df(S0_glc, alpha_df,gene_pairs, df_S0_glc, 'S0_glc',obj_div_df, scale_alpha,slim_opt)
    if not slim_opt:    
        df_E0 = df_E0.set_index('Gene_inhibition')
        df_S0_ac = df_S0_ac.set_index('Gene_inhibition')
        df_S0_glc = df_S0_glc.set_index('Gene_inhibition')    
        add_prefix_div_df(df_E0, 'E0')
        add_prefix_div_df(df_S0_ac, 'S0_ac')
        add_prefix_div_df(df_S0_glc,'S0_glc') 
    return([df_E0, df_S0_ac,df_S0_glc])


In [138]:
# # !!! obj_div_df_50percent-->alpha df directly
# ->genes_dict-> gene_pair
# div_df13 = find_gene_pair_inhibition(obj_div_df_50percent, 
#                                    genes_dict, minimum_NG_alpha = 1.02, scale_alpha=[1,3])

# div_df12 = find_gene_pair_inhibition(obj_div_df_50percent, 
#                                    genes_dict, minimum_NG_alpha = 1.02, scale_alpha=[1,2])



In [ ]:

div_df23 = find_gene_pair_inhibition(obj_div_df_50percent, 
                                   genes_dict, minimum_NG_alpha = 1.02, scale_alpha=[2,3])


In [ ]:
# conv_csv(div_df13,13)
conv_csv(div_df12,12)
conv_csv(div_df23,23)

In [ ]:

find_gene_pair_inhibition(obj_div_df_50percent, 
                                   genes_dict, minimum_NG_alpha = 1.02, scale_alpha=[2,3])

In [ ]:
def conv_csv(div_df, filelabel):
    E0_div_DG = div_df[0]
    S0_ac_div_DG = div_df[1]
    S0_glc_div_DG = div_df[2]

    def add_synergy_label(df, col): # dominant, synergistic, oppression 
        eval_df = pd.DataFrame()
        eval_df['max'] = df[[f"{col}_First_gene_inhibition_50p_obj_val", f"{col}_Second_gene_inhibition_50p_obj_val"]].max(axis=1)
        eval_df['min'] = df[[f"{col}_First_gene_inhibition_50p_obj_val", f"{col}_Second_gene_inhibition_50p_obj_val"]].min(axis=1)

        synergy_cond = eval_df['min'].round(5) > df[f"{col}_div_opt_obj_val"].round(5) # Synergy
        antagonism_cond = ((eval_df['max'].round(5) >= df[f"{col}_div_opt_obj_val"].round(5)) & 
                            (eval_df['min'].round(5) < df[f"{col}_div_opt_obj_val"].round(5)))
        dominance_cond = (eval_df['min'].round(5) == df[f"{col}_div_opt_obj_val"].round(5)) # correct for floating point precision 


        eval_df['Response'] = np.where(synergy_cond, 'Synergy', 
                                       np.where(dominance_cond, 'Dominance',
                                               np.where(antagonism_cond, 'Antagonism','Antagonism GT highest Single inhibition')
                                               )
                                      )
        df.insert(0,f"{col}_Drug_Response",eval_df['Response'])

    df1 = E0_div_DG.copy()
    add_synergy_label(df1, 'E0')
    df2 = S0_ac_div_DG.copy()
    add_synergy_label(df2, 'S0_ac')
    df3 = S0_glc_div_DG.copy()
    add_synergy_label(df3, 'S0_glc')

    df_full = pd.concat([df1,df2,df3],axis=1)
    df_full

    df_full.to_csv(f'DG_Div_detailed{filelabel}.csv')

    df_brief = df_full[['E0_Drug_Response','E0_div_opt_obj_val', 
                       'S0_ac_Drug_Response', 'S0_ac_div_opt_obj_val',
                      'S0_glc_Drug_Response','S0_glc_div_opt_obj_val']]
    df_brief.to_csv(f'DG_Div_brief{filelabel}.csv')
    return(df_full)
    
df = conv_csv(div_df12,12)

In [ ]:
df['E0_Drug_Response'].value_counts()

In [ ]:
k = pd.read_csv('ko_genes.csv')
k.shape

In [ ]:
pair_list = [ele for ele in itertools.combinations(list(genes_dict['higher_alpha']),2)]
from random import sample
# pair_list = sample(pair_list,20)
len(pair_list)

In [ ]:
scale_alpha_list = [(i,j) for i in reversed(np.linspace(1/5,1,5)) for j in np.linspace(1/5,1,5)]

scale_alpha_list = [(i,j) for i in np.linspace(2/5,4/5,3) for j in np.linspace(2/5,4/5,3)]
# for gene_pair in itertools.combinations(list(genes_dict['higher_alpha'])[:5],2):
for gene_pair in sample(pair_list,20):
    df = find_gene_pair_inhibition_alpha_val(obj_div_df_50percent, 
                                       gene_pair, minimum_NG_alpha = 1.02, scale_alpha_list=scale_alpha_list)
    a = np.array(df['div_opt_obj_val']).reshape(3,3)
    fig, ax = plt.subplots()
    ax.set_title(f'{gene_pair}')
    im = ax.imshow(a)
    


In [ ]:
df['div_opt_obj_val']

In [ ]:
# plt.matshow(df['div_opt_obj_val'],1)
scale_alpha_list